# Humanizing Customer Complaints 


### **Problem Defination:** a supervised (labelled) text multi-class classification problem, of which our goal is to make a prediction (assign to correct category) with a new input (complaint).

### Kaggle auth token file upload(seperately downloaded from kaggle account)

In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"koushiktcs12","key":"d8bf4887882fc2016aa969ef44d835c9"}'}

## Copy uploaded kaggle.json file into google colab woking directory.

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                     title                                              size  lastUpdated          downloadCount  
------------------------------------------------------  -----------------------------------------------  ------  -------------------  -------------  
unanimad/us-election-2020                               US Election 2020                                  429KB  2020-11-19 10:31:50           6543  
antgoldbloom/covid19-data-from-john-hopkins-university  COVID-19 data from John Hopkins University          2MB  2020-11-19 06:04:10           2854  
manchunhui/us-election-2020-tweets                      US Election 2020 Tweets                           353MB  2020-11-09 18:51:59           2945  
headsortails/us-election-2020-presidential-debates      US Election 2020 - Presidential Debates           199MB  2020-10-23 16:56:10            525  
etsc9287/2020-general-election-polls                    Election, COVID, and Demographic Data by Cou

## Download pre-trained word embedding dictionary from kaggle

In [3]:
!kaggle datasets download -d watts2/glove6b50dtxt
!kaggle datasets download -d danielwillgeorge/glove6b100dtxt
!kaggle datasets download -d incorpes/glove6b200d
!kaggle datasets download -d thanakomsn/glove6b300dtxt
!kaggle datasets download -d leadbest/googlenewsvectorsnegative300
!unzip \*.zip  && rm *.zip
!ls

 69% 47.0M/67.7M [00:00<00:00, 35.0MB/s]
100% 67.7M/67.7M [00:01<00:00, 69.8MB/s]
 93% 122M/131M [00:01<00:00, 74.1MB/s]
100% 131M/131M [00:01<00:00, 69.5MB/s]
100% 258M/259M [00:03<00:00, 50.5MB/s]
100% 259M/259M [00:03<00:00, 76.4MB/s]
 98% 377M/386M [00:05<00:00, 47.9MB/s]
100% 386M/386M [00:05<00:00, 71.8MB/s]
100% 3.17G/3.17G [00:45<00:00, 126MB/s]
100% 3.17G/3.17G [00:45<00:00, 74.6MB/s]
Archive:  googlenewsvectorsnegative300.zip
  inflating: GoogleNews-vectors-negative300.bin  
  inflating: GoogleNews-vectors-negative300.bin.gz  

Archive:  glove6b100dtxt.zip
  inflating: glove.6B.100d.txt       

Archive:  glove6b300dtxt.zip
  inflating: glove.6B.300d.txt       

Archive:  glove6b50dtxt.zip
  inflating: glove.6B.50d.txt        

Archive:  glove6b200d.zip
  inflating: glove.6B.200d.txt       

5 archives were successfully processed.
glove.6B.100d.txt  glove.6B.50d.txt			  kaggle.json
glove.6B.200d.txt  GoogleNews-vectors-negative300.bin	  sample_data
glove.6B.300d.txt  GoogleNew

## Installation of required packages before import in colab

In [4]:
!pip install spacy-langdetect
!pip install language-detector
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')

     |████████████████████████████████| 1.0MB 8.8MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993461 sha256=6529c25e430b519cc6728128e11d51bba514b85c9e8bd47de79975598fdf5cc3
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect
  Created wheel for language-detector: filename=language_detector-5.0.2-cp36-none-any.whl size=7055 sha256=62ca451af1885e291b56a7c7b59f6b47fa905633650e9960e59ee1fa83c04dce
  Stored in directory: /root/.cache/pip/wheels/7d/37/fa/2098a4aa6c0d94d6ddff0d3a79669e12bc4f7baca8a760b3db
Successfully built language-detector
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...


True

# Section 1: Importing Packages and Loading Data

In [5]:
# Import libraries
# This is used for fast string concatination
from io import StringIO
import pandas as pd
pd.options.display.max_colwidth = 1000
import os
import time
import datetime
from tqdm.gui import trange, tqdm
from pprint import pprint
from matplotlib import pyplot as plt
import re
import nltk
import numpy as np
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import spacy
nlp=spacy.load('en_core_web_sm')
from gensim import corpora, models
import gensim
from nltk.tag import pos_tag
import collections as co
from collections import OrderedDict
import pickle # for saving and loading objects
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Classifiers
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

# Metrics to score classifiers
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve, log_loss

# Data splitting, CV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

# Lifesaver
from google.colab import files
seed =300

### **Data Source** Kaggle https://www.kaggle.com/cfpb/us-consumer-finance-complaints
### Original data source: https://files.consumerfinance.gov/ccdb/complaints.csv.zip

## Load the input complaints text dataset(CFPB data from kaggle)

In [6]:
!kaggle datasets download -d cfpb/us-consumer-finance-complaints
!kaggle datasets download -d koushiktcs12/cfpb-complaints
!unzip \*.zip  && rm *.zip
!ls

 85% 72.0M/84.5M [00:00<00:00, 58.2MB/s]
100% 84.5M/84.5M [00:01<00:00, 87.1MB/s]
 94% 267M/285M [00:03<00:00, 51.0MB/s]
100% 285M/285M [00:03<00:00, 78.4MB/s]
Archive:  cfpb-complaints.zip
  inflating: complaints.csv          

Archive:  us-consumer-finance-complaints.zip
  inflating: consumer_complaints.csv  
  inflating: database.sqlite         

2 archives were successfully processed.
complaints.csv		 glove.6B.50d.txt
consumer_complaints.csv  GoogleNews-vectors-negative300.bin
database.sqlite		 GoogleNews-vectors-negative300.bin.gz
glove.6B.100d.txt	 kaggle.json
glove.6B.200d.txt	 sample_data
glove.6B.300d.txt


In [7]:
# # Load CSV file
df1= pd.read_csv(r'consumer_complaints.csv')
# This step is not executable because the .csv file is not in the drive
df2= pd.read_csv(r'complaints.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
# Column Cleaning First. Columns are super nastily named
df2.rename(columns={'Date received':'DATE_RECEIVED',
                  'Product':'PRODUCT',
                  'Sub-product':'SUB_PRODUCT',
                  'Issue':'ISSUE',
                  'Sub-issue':'SUB_ISSUE',
                  'Consumer complaint narrative':'CONSUMER_COMPLAINT_NARRATIVE',
                  'Company public response':'COMPANY_PUBLIC_RESPONSE',
                   'Company':'COMPANY',
                   'State':'STATE',
                   'ZIP code':'ZIP_CODE',
                   'Tags':'TAGS',
                   'Consumer consent provided?':'CONSUMER_CONSENT_PROVIDED',
                   'Subbmited via':'SUBMITTED_VIA',
                   'Date sent to company':'DATE_SENT_TO_COMPANY',
                   'Company response to consumer':'COMPANY_RESPONSE_TO_CONSUMER',
                   'Timely response?':'TIMELY_RESPONSE',
                   'Consumer disputed?':'CONSUMER_DISPUTED',
                   'Complaint ID':'COMPLAINT_ID'
                  }, inplace=True)

                  # Column Cleaning First. Columns are super nastily named
df1.rename(columns={'date_received':'DATE_RECEIVED',
                  'product':'PRODUCT',
                  'sub_product':'SUB_PRODUCT',
                  'issue':'ISSUE',
                  'sub_issue':'SUB_ISSUE',
                  'consumer_complaint_narrative':'CONSUMER_COMPLAINT_NARRATIVE',
                  'company_public_response':'COMPANY_PUBLIC_RESPONSE',
                   'company':'COMPANY',
                   'state':'STATE',
                   'zipcode':'ZIP_CODE',
                   'tags':'TAGS',
                   'consumer_consent_provided':'CONSUMER_CONSENT_PROVIDED',
                   'submitted_via':'SUBMITTED_VIA',
                   'date_sent_to_company':'DATE_SENT_TO_COMPANY',
                   'company_response_to_consumer':'COMPANY_RESPONSE_TO_CONSUMER',
                   'timely_response':'TIMELY_RESPONSE',
                   'consumer_disputed?':'CONSUMER_DISPUTED',
                   'complaint_id':'COMPLAINT_ID'
                  }, inplace=True)

In [9]:
print(df1.shape)
print(df2.shape)
frames = [df1, df2]
df = pd.concat(frames)
print(df.info())

(555957, 18)
(1835193, 18)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2391150 entries, 0 to 1835192
Data columns (total 19 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   DATE_RECEIVED                 object
 1   PRODUCT                       object
 2   SUB_PRODUCT                   object
 3   ISSUE                         object
 4   SUB_ISSUE                     object
 5   CONSUMER_COMPLAINT_NARRATIVE  object
 6   COMPANY_PUBLIC_RESPONSE       object
 7   COMPANY                       object
 8   STATE                         object
 9   ZIP_CODE                      object
 10  TAGS                          object
 11  CONSUMER_CONSENT_PROVIDED     object
 12  SUBMITTED_VIA                 object
 13  DATE_SENT_TO_COMPANY          object
 14  COMPANY_RESPONSE_TO_CONSUMER  object
 15  TIMELY_RESPONSE               object
 16  CONSUMER_DISPUTED             object
 17  COMPLAINT_ID                  int64 
 18  Submitted via  

### Section 1.0.1: Dropping all rows that do not have Customer Complaint entries in them

In [10]:
df = df[df["CONSUMER_COMPLAINT_NARRATIVE"].notnull() == True]
df = df.drop_duplicates()
df['Text_Length'] = df["CONSUMER_COMPLAINT_NARRATIVE"].str.len()

## Subsetting source data columns for analysis

In [11]:
# Subsetting dataframe into columns useful for our text multi-classification problem
df_product_and_complaint = df[['PRODUCT','ISSUE', 'CONSUMER_COMPLAINT_NARRATIVE','COMPLAINT_ID']]
# Renaming columns to something easier
df_product_and_complaint.rename(columns={'CONSUMER_COMPLAINT_NARRATIVE':'CONSUMER_COMPLAINT'},inplace=True) 

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## Dropping multilevel product category data

In [12]:
# Dropping rows with the aggregated columns
df_product_and_complaint.drop(
    df_product_and_complaint[
    df_product_and_complaint.PRODUCT == 
    'Credit reporting, credit repair services, or other personal consumer reports'].index, 
    inplace=True) # credit_aggregated

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## Merging all diffrent type of loans category into single 'loan' category
### Rename multinaming level to most appropriate level name

In [13]:
## Combining Loans together to make this a simplier classification problem
df_product_and_complaint.replace('Student loan', 'Loan', inplace=True)
df_product_and_complaint.replace('Consumer Loan', 'Loan', inplace=True)
df_product_and_complaint.replace('Payday loan', 'Loan', inplace=True)
df_product_and_complaint.replace('Vehicle loan or lease', 'Loan', inplace=True)
df_product_and_complaint.replace('Credit card or prepaid card', 'Credit card', inplace=True)
df_product_and_complaint.replace('Payday loan, title loan, or personal loan', 'Loan', inplace=True)
df_product_and_complaint.replace('Money transfer, virtual currency, or money service', 'Money transfers', inplace=True)
# Placing Virtual currency into other financial service
df_product_and_complaint.replace('Virtual currency', 'Other financial service', inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [14]:
print(df_product_and_complaint['PRODUCT'].value_counts())

Debt collection                139993
Mortgage                        82530
Credit card                     70060
Loan                            62830
Credit reporting                41404
Checking or savings account     26195
Bank account or service         19386
Money transfers                 13537
Prepaid card                     2104
Other financial service           390
Name: PRODUCT, dtype: int64


### We have a wooping 458,429 entries in our dataframe.
#### This might cause our comptuer to crash. So we gotta reduce it.

In [76]:
# Reducing dataframe to 36k entries
# Need to reduce top 8 categories by 80% and for other 2 kept remain same

percentage_reduction_0 = 0.97
percentage_reduction_1 = 0.95
percentage_reduction_2 = 0.0

# Debt Collection Reduction
df_product_and_complaint_reduced = \
df_product_and_complaint.drop(
  df_product_and_complaint[df_product_and_complaint['PRODUCT'] == 'Debt collection'].sample(frac=percentage_reduction_0).index)

# Mortgage Reduction
df_product_and_complaint_reduced = \
df_product_and_complaint_reduced.drop(
  df_product_and_complaint_reduced[
      df_product_and_complaint_reduced['PRODUCT'] == 'Mortgage'].sample(frac=percentage_reduction_1).index)

# Loan
df_product_and_complaint_reduced = \
df_product_and_complaint_reduced.drop(
  df_product_and_complaint_reduced[
      df_product_and_complaint_reduced['PRODUCT'] == 'Loan'].sample(frac=percentage_reduction_1).index)

# Credit reporting
df_product_and_complaint_reduced = \
df_product_and_complaint_reduced.drop(
  df_product_and_complaint_reduced[
      df_product_and_complaint_reduced['PRODUCT'] == 'Credit reporting'].sample(frac=percentage_reduction_1).index)

# Credit card
df_product_and_complaint_reduced = \
df_product_and_complaint_reduced.drop(
  df_product_and_complaint_reduced[
      df_product_and_complaint_reduced['PRODUCT'] == 'Credit card'].sample(frac=percentage_reduction_1).index)

# Checking or savings account
df_product_and_complaint_reduced = \
df_product_and_complaint_reduced.drop(
  df_product_and_complaint_reduced[
      df_product_and_complaint_reduced['PRODUCT'] == 'Checking or savings account'].sample(frac=percentage_reduction_1).index)

# Bank account or service
df_product_and_complaint_reduced = \
df_product_and_complaint_reduced.drop(
    df_product_and_complaint_reduced[
        df_product_and_complaint_reduced['PRODUCT'] == 'Bank account or service'].sample(frac=percentage_reduction_1).index)

#Money transfers 
df_product_and_complaint_reduced = \
df_product_and_complaint_reduced.drop(
    df_product_and_complaint_reduced[
        df_product_and_complaint_reduced['PRODUCT'] == 'Money transfers'].sample(frac=percentage_reduction_1).index)

# Prepaid card (too little to be reduced)
df_product_and_complaint_reduced = \
df_product_and_complaint_reduced.drop(
    df_product_and_complaint_reduced[
        df_product_and_complaint_reduced['PRODUCT'] == 'Prepaid card'].sample(frac=percentage_reduction_2).index)

# Other financial service (too little to be reduced)
df_product_and_complaint_reduced = \
df_product_and_complaint_reduced.drop(
    df_product_and_complaint_reduced[
        df_product_and_complaint_reduced['PRODUCT'] == 'Other financial service'].sample(frac=percentage_reduction_2).index)


In [77]:
print(df_product_and_complaint_reduced['PRODUCT'].value_counts())

Debt collection                3966
Mortgage                       3899
Credit card                    3304
Loan                           3012
Prepaid card                   1967
Credit reporting               1943
Checking or savings account    1239
Bank account or service         917
Money transfers                 633
Other financial service         364
Name: PRODUCT, dtype: int64


In [78]:
df_product_and_complaint_reduced.index = df_product_and_complaint_reduced.COMPLAINT_ID
df = df_product_and_complaint_reduced
df = df.rename(columns={"CONSUMER_COMPLAINT": "Complain_text"})
df = df.set_index('COMPLAINT_ID')
df['COMPLAINT_ID'] = df.index
complain_text = df[['Complain_text']]
texts = complain_text.drop_duplicates()
df_texts = pd.DataFrame(texts,columns=['Complain_text'])
df_texts = df_texts[df_texts['Complain_text'].notnull() == True]

In [79]:
sp = spacy.load('en_core_web_sm')
all_stopwords_sp = sp.Defaults.stop_words
print(type(all_stopwords_sp))

<class 'set'>


In [80]:
all_stopwords_gn = gensim.parsing.preprocessing.STOPWORDS
print(type(all_stopwords_gn))

<class 'frozenset'>


In [81]:
all_stopwords=set(stopwords.words('english'))|all_stopwords_sp|all_stopwords_gn
print(len(all_stopwords))

413


## Text cleansing functions defination

In [82]:
nlp=spacy.load('en_core_web_sm')

#remove_stop_words()  
def removeStopwords():
  global text
  sp = spacy.load('en_core_web_sm')
  all_stopwords_sp = sp.Defaults.stop_words
  all_stopwords_gn = gensim.parsing.preprocessing.STOPWORDS
  all_stopwords_comb = all_stopwords_sp|all_stopwords_gn
  text_tokens = word_tokenize(text)
  #print('text_tokens',text_tokens)
  tokens_without_sw= [word for word in text_tokens if not word in all_stopwords_comb]  
  text = (" ").join(tokens_without_sw)  

def removeUnicode():
    global text
    """ Removes unicode strings like "\u002c" and "x96" """
    text = re.sub(r'(\\u[0-9A-Fa-f]+)',r' ', text)       
    text = re.sub(r'[^\x00-\x7f]',r' ',text)

def replaceURL():
    global text
    """ Replaces url address with "url" """
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
#    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    text = text.replace("-","").replace(".com","")

def replaceAtUser():
    global text
    """ Replaces "@user" with "atUser" """
    text = re.sub('@[^\s]+',' ',text)

def removeHashtagInFrontOfWord():
    global text
    """ Removes hastag in front of a word """
    text = re.sub(r'#([^\s]+)', r'\1', text)

def removeNumbers():
    global text
    """ Removes integers """
    text = ''.join([i for i in text if not i.isdigit()])         

def replaceMultiExclamationMark():
    global text
    """ Replaces repetitions of exlamation marks """
#    text = re.sub(r"(\!)\1+", ' multiExclamation ', text)
    text = re.sub(r"(\!)\1+", ' ', text)

def replaceMultiQuestionMark():
    global text
    """ Replaces repetitions of question marks """
#    text = re.sub(r"(\?)\1+", ' multiQuestion ', text)
    text = re.sub(r"(\?)\1+", ' ', text)

def replaceMultiStopMark():
    global text
    """ Replaces repetitions of stop marks """
#    text = re.sub(r"(\.)\1+", ' multiStop ', text)
    text = re.sub(r"(\.)\1+", '.', text)
    
def remove_special_characters():
    global text
    '''
    Function to clean the text in a tweet by removing 
    links and special characters with regex.
    '''
    text = ' '.join(re.sub("(@[A-Za-z[0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\/S+)", " ", text).split())
    
def tokenize_text():
    global text
    global tokens
    tokens=text.split()
#    tokens = nltk.word_tokenize(text)

def stemming_text():
    global tokens
    ps_stem = PorterStemmer()
    for i in range(len(tokens)):
        tokens[i] = ps_stem.stem(tokens[i])

def lemmatize_text():
    global text
    global tokens
    
    tokens=text.split()
    lemma = WordNetLemmatizer()
    
    for t in tokens:
        t1 = lemma.lemmatize(t)
        t1 = " " + str(t1) + " "
        text=text.replace(t,t1)
      
def smart_lemmatize():
        global text
        global tokens
        
        """Detects the part of speech for each word and then lemmatizes using that tag if available (default to Noun)
       Returns
       -------
       list
           list of lemmatized words
       Examples
       --------
       >>> self.smart_lemmatize(['testing','this','out'])
       ['test','this','out']
        """
        new_sentence = []
        lemma = WordNetLemmatizer()
        #print(text)
        text_split=text.split(" ")

        tagged_sentence = pos_tag([word for word in text_split if word])
        #print(tagged_sentence)
        for word, tag in tagged_sentence:
            if tag in ['NN', 'NNS', 'NNP', 'NNPS']:
                pos = 'n'
            elif tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
                pos = 'v'
            elif tag in ['JJ', 'JJR', 'JJS']:
                pos = 'a'
            elif tag in ['RB', 'RBR', 'RBS']:
                pos = 'r'
            else:
                pos = 'n'
            lemma_word = lemma.lemmatize(word=word, pos=pos)
            
            new_sentence.append(lemma_word)

        #text=new_sentence
        text2=""
        for i in new_sentence:
            text2 = text2 + " " + i
        text=text2
        #print(text)

def remove_names():
        global text,nlp
        new_sentence = []
        #print(text)        
        doc=nlp(text)
        for ent in doc.ents:
          #print(ent.text,ent.label_)
          text = text.replace(ent.text,"")


## Text Preprocessing steps

In [127]:
def clean_text_ngram(text1):
    global text
    global tokens
    
    text = text1
    remove_names()
    text=text.lower()
    removeStopwords()      
    text=" " + text + " "
    #stemming_text()
    #lemmatize_text()
    #replace_incomplete_word()
    smart_lemmatize()
    removeUnicode()
    replaceURL()
    replaceAtUser()
    removeHashtagInFrontOfWord()
    removeNumbers()    
    replaceMultiExclamationMark()
    replaceMultiQuestionMark()
    replaceMultiStopMark()
    remove_special_characters()
    text1 = text   
    return text1

   
def tokenize_text(df_test):    
    global texts_cln,texts_mod

    texts_cln=[]
    texts_mod=[]
    i=0
    
    for text1 in df_test.Complain_text:        
        text1=clean_text_ngram(text1)
        i=i+1
        print(i)
        s=StringIO()
        s.write(text1)
        k=s.getvalue()
        s.close()
        k=k.lower()
        k=k.split()        
        # Next only want valid strings
        words = co.Counter(nltk.corpus.words.words())
        k=[i for i in k if i in words]
        s=" ".join(k)
        c = co.Counter(k)    
        texts_mod.append(text1)

## Cleansing input complaint text

In [ ]:
df_texts = df_texts.reset_index(drop=True)
tokenize_text(df_texts)
df_mod=pd.DataFrame(texts_mod,columns=['modified_text'])
df_mod.index = df_texts.index
df_all=df_texts
df_all=df_all.join(df_mod)
df_all = df_all[df_all['modified_text'].notnull() == True]
df_all['COMPLAINT_ID'] = df_all.index
df_all = df_all[['COMPLAINT_ID', 'Complain_text', 'modified_text']].rename(columns={'COMPLAINT_ID':'Complaint_ID','modified_text':'Modified_text'})
title_name="CFPB_text_vs_cnsd_text"
dt_stamp = datetime.datetime.strftime(datetime.datetime.now(),'%Y%m%d_%H_%M_%S')
file_name=str(title_name)+str("_")+str(dt_stamp)+ str(".xlsx")
df_product_and_complaint_reduced = df_product_and_complaint_reduced.join(df_all[['Modified_text']])
df_product_and_complaint_reduced.to_excel(file_name, header=True)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## Product level data distribution

In [123]:
df = df_product_and_complaint_reduced[['PRODUCT','ISSUE','COMPLAINT_ID','Modified_text']].rename(columns={'Modified_text':'CONSUMER_COMPLAINT'})
print(df[['PRODUCT']].value_counts(normalize=True)*100)

PRODUCT                    
Debt collection                27.663230
Mortgage                       16.247590
Credit card                    13.846283
Loan                           12.610008
Prepaid card                    8.243232
Credit reporting                8.176180
Checking or savings account     5.183975
Bank account or service         3.838739
Money transfers                 2.661135
Other financial service         1.529629
dtype: float64


### Section 1.1.2: Subsetting Dataframe for Text Multi-Classification Problem

In [ ]:
# Subsetting dataframe into columns useful for our text multi-classification problem
df = df[['PRODUCT','CONSUMER_COMPLAINT']].copy()
df.CONSUMER_COMPLAINT = df.CONSUMER_COMPLAINT.astype(str)

In [ ]:
# Pickling our subsetted dataframe
with open('df_data.pickle', 'wb') as to_write:
     pickle.dump(df, to_write)

In [ ]:
# Loading our pickled subsetted dataframe
with open('df_data.pickle', 'rb') as to_read:
    df_dc = pickle.load(to_read)

### Section 2.2.4: Label Encoding for Issue Categories

In [ ]:
# Applying encoding to the ISSUE column
df['PRODUCT_ID'] = df['PRODUCT'].factorize()[0] 

#.factorize[0] arranges the index of each encoded number accordingly to the 
# index of your categorical variables in the ISSUE column

# Creates a dataframe of the ISSUE to their respective ISSUE_ID
category_id_df = df[['PRODUCT', 'PRODUCT_ID']].drop_duplicates()

# Dictionaries for future use. Creating our cheatsheets for what each encoded label represents.
category_to_id = dict(category_id_df.values) # Creates a ISSUE: ISSUE_ID key-value pair
id_to_category = dict(category_id_df[['PRODUCT_ID', 'PRODUCT']].values)  # Creates a ISSUE_ID: ISSUE key-value pair

# New dataframe
df_dc.head(10)

Ok so basically, we can trust the stop_word removal from tfidf.   
Through the tests, we see that it removes punctuations and also all the stopwords.   
(I deleted these because they were seriously taking up a lot of space on my jupyter notebook)  
Spellchecker might also mess up some words, especially with bank names like Citi for example  
Thus changing the semantic meaning of the word.  
  
Therefore, we are going to just use tfidf just as it is.  
It's pretty amazing, really.  
  
Great! We can start building some shit!  

# Section 3: Model/Classifier Selection
#### Each model is cross-validated on a stratified 5-fold split train-validation (of 80% of the whole dataset). The remaining 20% data was held out and untouched to simulate how our final chosen model would perform in real-life scenarios

## Section 3.1: Train/StratifiedCV/Test split

In this section, we also prepare the model for text pre-processing. This is handled by our handydandy Tfidfvecotirzer with the stop_words and such. As we have tested above, it also ignores punctuation so it really is quite a catch-all package that cleans our text really well for the purposes of doing text classification

In [ ]:
# Split the data into X and y data sets
X, y = df.CONSUMER_COMPLAINT, df.PRODUCT_ID
print('X shape:', X.shape, 'y shape:', y.shape)

# For text classification, ALWAYS split data first before vectorizing.
# This is because you don't want to cheat by having features (words) from the test data already being inside your train data
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, 
                                                            test_size=0.2,   # 80% train/cv, 20% test
                                                            stratify=y,
                                                            random_state=seed)
print('X_train', X_train_val.shape)
print('y_train', y_train_val.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

In [ ]:
X.isnull().sum()

## Section 2.2: Vectorization
#### To turn words into something a machine learning algorithm can understand and process, we need to do something called vectorization.
##### Put simply, this is the process of turning words into multi-dimensional vectors in such a way that the meaning or context of the word is correlated with where that vector points. In a sense, vectorization allows computers to quantify the meaning of words by mapping similar word meanings to similar vector spaces.

In [ ]:
# Performing Text Pre-Processing

# Import tfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# The text needs to be transformed to vectors so as the algorithms will be able make predictions. 
# In this case it will be used the Term Frequency – Inverse Document Frequency (TFIDF) weight 
# to evaluate how important A WORD is to A DOCUMENT in a COLLECTION OF DOCUMENTS.

# tfidf1 = 1-gram only. 
tfidf1 = TfidfVectorizer(sublinear_tf=True, # set to true to scale the term frequency in logarithmic scale.
                        min_df=5,
                        stop_words='english')

X_train_val_tfidf1 = tfidf1.fit_transform(X_train_val).toarray()
X_test_tfidf1 = tfidf1.transform(X_test)

# tfidf2 = unigram and bigram
tfidf2 = TfidfVectorizer(sublinear_tf=True, # set to true to scale the term frequency in logarithmic scale.
                        min_df=5, 
                        ngram_range=(1,2), # we consider unigrams and bigrams
                        stop_words='english')
X_train_val_tfidf2 = tfidf2.fit_transform(X_train_val).toarray()
X_test_tfidf2 = tfidf2.transform(X_test)

# # StratifiedKFold -> Split 5
# ## We now want to do stratified kfold to preserve the proportion of the category imbalances 
# # (number is split evenly from all the classes)

## Term Frequency — Inverse Document Frequency Vectorization
#### For this application, we want context-based vectorization. This is where Term frequency inverse document frequency vectorization comes in (TF-IDF). This vectorization method looks at the number of times a word appears in a comment relative to the number of times it appears in other comments. Two things result in a higher TF-IDF score:

   1. Higher frequency of the word within the specific complaint being scored.
   2. Lower frequency of the word across all other complaints.

## Section 3.2: Baseline Model - Train/Stratified CV with MultinomialNB()

In [ ]:
print('1-gram number of (rows, features):', X_train_val_tfidf1.shape)

### Evaluation Metrics (Accuracy, Recall_macro, Precission_macro, f1_macro, Recall_weighted, Precission_weighted, f1_weighted)
Referrence:
https://towardsdatascience.com/multi-class-metrics-made-simple-part-i-precision-and-recall-9250280bddc2

https://towardsdatascience.com/multi-class-metrics-made-simple-part-ii-the-f1-score-ebe8b2c2ca1


In [ ]:
def metric_cv_stratified(model, X_train_val, y_train_val, n_splits, name):
    """
    Accepts a Model Object, converted X_train_val and y_train_val, n_splits, name
    and returns a dataframe with various cross-validated metric scores 
    over a stratified n_splits kfold for a multi-class classifier.
    """
    # Start timer
    import timeit
    start = timeit.default_timer()
    
    ### Computations below
    
    # StratifiedKFold
    ## We now want to do stratified kfold to preserve the proportion of the category imbalances 
    # (number is split evenly from all the classes)
    from sklearn.model_selection import StratifiedKFold  # incase user forgest to import
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    
    # Initializing Metrics
    accuracy = 0.0
    micro_f1 = 0.0
    macro_precision = 0.0
    macro_recall = 0.0
    macro_f1 = 0.0
    weighted_precision = 0.0
    weighted_recall = 0.0
    weighted_f1 = 0.0
#     roc_auc = 0.0    Not considering this score in this case
        
    # Storing metrics
    from sklearn.model_selection import cross_val_score  # incase user forgets to import
    accuracy = np.mean(cross_val_score(model, X_train_val, y_train_val, cv=kf, scoring='accuracy'))
#     micro_f1 = np.mean(cross_val_score(model, X_train_val, y_train_val, cv=kf, scoring='f1_micro'))
    macro_precision = np.mean(cross_val_score(model, X_train_val, y_train_val, cv=kf, scoring='precision_macro'))
    macro_recall = np.mean(cross_val_score(model, X_train_val, y_train_val, cv=kf, scoring='recall_macro'))
    macro_f1 = np.mean(cross_val_score(model, X_train_val, y_train_val, cv=kf, scoring='f1_macro'))
    weighted_precision = np.mean(cross_val_score(model, X_train_val, y_train_val, cv=kf, scoring='precision_weighted'))
    weighted_recall = np.mean(cross_val_score(model, X_train_val, y_train_val, cv=kf, scoring='recall_weighted'))
    weighted_f1 = np.mean(cross_val_score(model, X_train_val, y_train_val, cv=kf, scoring='f1_weighted'))
    
    # Stop timer
    stop = timeit.default_timer()
    elapsed_time = stop - start
    
    return pd.DataFrame({'Model'    : [name],
                         'Accuracy' : [accuracy],
#                          'Micro F1' : [micro_f1],
                         'Macro Precision': [macro_precision],
                         'Macro Recall'   : [macro_recall],
                         'Macro F1score'  : [macro_f1],
                         'Weighted Precision': [weighted_precision],
                         'Weighted Recall'   : [weighted_recall],
                         'Weighted F1'  : [weighted_f1],
                         'Time taken': [elapsed_time]  # timetaken: to be used for comparison later
                        })

## Section 3.3: Further Train/Straitified CV with other Models

### Section 3.3.1: 1-gram

In [ ]:
## Testing on all Models using 1-gram 

# Initialize Model Object
gnb = GaussianNB()
mnb = MultinomialNB()
logit = LogisticRegression(random_state=seed)
randomforest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
dtc = DecisionTreeClassifier(max_depth = 10, random_state = 20)
pac = PassiveAggressiveClassifier()
xgb = XGBClassifier(max_depth = 7, n_estimators = 50)
linearsvc = LinearSVC()

## We do NOT want these two. They take FOREVER to train AND predict
# knn = KNeighborsClassifier()  
# decisiontree = DecisionTreeClassifier(random_state=seed)

# to concat all models
results_cv_straitified_1gram = pd.concat([metric_cv_stratified(mnb, X_train_val_tfidf1, y_train_val, 5, 'MultinomialNB1'),
                                           metric_cv_stratified(gnb, X_train_val_tfidf1, y_train_val, 5, 'GaussianNB1'),
                                           metric_cv_stratified(logit, X_train_val_tfidf1, y_train_val, 5, 'LogisticRegression1'),
                                           metric_cv_stratified(randomforest, X_train_val_tfidf1, y_train_val, 5, 'RandomForest1'),
                                           metric_cv_stratified(dtc, X_train_val_tfidf1, y_train_val, 5, 'DecisionTreeClassifier1'),
                                           metric_cv_stratified(pac, X_train_val_tfidf1, y_train_val, 5, 'PassiveAggressiveClassifier1'),
                                           metric_cv_stratified(xgb, X_train_val_tfidf1, y_train_val, 5, 'XGBClassifier1'),
                                           metric_cv_stratified(linearsvc, X_train_val_tfidf1, y_train_val, 5, 'LinearSVC1')
                                          ], axis=0).reset_index()

In [ ]:
results_cv_straitified_1gram

### Logistic Regression gives us the highest accuracy of 39%.

In [ ]:
# Saving our results because I don't wanna re-train the whole damn thing
with open('results_cv_straitified_1gram_df.pickle', 'wb') as to_write:
  pickle.dump(results_cv_straitified_1gram, to_write)

In [ ]:
# Loading our pickled results
with open('results_cv_straitified_1gram_df.pickle', 'rb') as to_read:
    results_cv_straitified_1gram = pickle.load(to_read)

### Section 3.3.2: 2-gram (Keep running into memory issues, therefore, this will not be tested anymore)

In [ ]:
### Keep running into memory issues with 2-gram. Therefore, will not test it anymore

'''# Testing on all Models using 2-gram 

# Initialize Model Object
gnb = GaussianNB()
mnb = MultinomialNB()
logit = LogisticRegression(random_state=seed)
knn = KNeighborsClassifier()
decisiontree = DecisionTreeClassifier(random_state=seed)
randomforest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
linearsvc = LinearSVC()

# # to concat all models
results_cv_straitified_2gram = pd.concat([metric_cv_stratified(mnb, X_train_val_tfidf2, y_train_val, 5, 'MultinomialNB2'),
                                          metric_cv_stratified(gnb, X_train_val_tfidf2, y_train_val, 5, 'GaussianNB2'),
                                          metric_cv_stratified(logit, X_train_val_tfidf2, y_train_val, 5, 'LogisticRegression2'),
                                          metric_cv_stratified(randomforest, X_train_val_tfidf2, y_train_val, 5, 'RandomForest2'),
                                          metric_cv_stratified(dtc, X_train_val_tfidf2, y_train_val, 5, 'DecisionTreeClassifier'),
                                          metric_cv_stratified(pac, X_train_val_tfidf2, y_train_val, 5, 'PassiveAggressiveClassifier'),
                                          metric_cv_stratified(xgb, X_train_val_tfidf2, y_train_val, 5, 'XGBClassifier'),
                                          metric_cv_stratified(linearsvc, X_train_val_tfidf2, y_train_val, 5, 'LinearSVC2')
                                          ], axis=0).reset_index()'''

In [ ]:
#results_cv_straitified_2gram

Instead of utilizing Vectorization (Tf-idfVectorizer that we used above), we can utilize WordEmbedding on the complaints before feeding them into our classifiers.
https://towardsdatascience.com/word2vec-skip-gram-model-part-1-intuition-78614e4d6e0b

https://towardsdatascience.com/the-three-main-branches-of-word-embeddings-7b90fa36dfb9

### Section 3.3.3: Using GloVe50d


Each complaint is mapped to a feature vector by averaging the word embeddings of all words in the review. These features are then fed into the defined function above for train/cross validation.

In [ ]:
## Using pre-trained GloVe 'glove.6B.50d'

glove_file = glove_dir = 'glove.6B.50d.txt'
w2v_output_file = 'glove.6B.50d.txt.w2v'

# The following utility converts file formats
gensim.scripts.glove2word2vec.glove2word2vec(glove_file, w2v_output_file)

# Now we can load it!
glove_model_50d = gensim.models.KeyedVectors.load_word2vec_format(w2v_output_file, binary=False)

# Pickle glove model so we don't have to do the above steps again and keep the damn glove.6b.50d in our folder
with open('glove_model_50d.pickle', 'wb') as to_write:
    pickle.dump(glove_model_50d, to_write)

In [ ]:
# Load pickled glove_model
with open('glove_model_50d.pickle', 'rb') as to_read:
    glove_model_50d = pickle.load(to_read)
    
num_features = 50 # depends on the pre-trained model you are loading

In [ ]:
def complaint_to_wordlist(review, remove_stopwords=False):
    """
    Convert a complaint to a list of words. Removal of stop words is optional.
    """
    # remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review)
    
    # convert to lower case and split at whitespace
    words = review_text.lower().split()
    
    # remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    return words    # list of tokenized and cleaned words

In [ ]:
# num_features refer to the dimensionality of the model you are using
# model refers to the trained word2vec/glove model
# words refer to the words in a single document/entry

def make_feature_vec(words, model, num_features):
    """
    Average the word vectors for a set of words
    """
    feature_vec = np.zeros((num_features,),  # creates a zero matrix of (num_features, )
                           dtype="float32")  # pre-initialize (for speed)
    
    # Initialize a counter for the number of words in a complaint
    nwords = 0.
    index2word_set = set(model.index2word)  # words known to the model

    
    # Loop over each word in the comment and, if it is in the model's vocabulary, add its feature vector to the total
    for word in words:   # for each word in the list of words
        if word in index2word_set:   # if each word is found in the words known to the model
            nwords = nwords + 1.     # add 1 to nwords
            feature_vec = np.add(feature_vec, model[word])   
    
    # Divide by the number of words to get the average 
    if nwords > 0:
        feature_vec = np.divide(feature_vec, nwords)
    
    return feature_vec

In [ ]:
# complaints refers to the whole corpus you intend to put in. 
# Therefore you need to append all these info from your df into a list first

def get_avg_feature_vecs(complaints, model, num_features):
    """
    Calculate average feature vectors for ALL complaints
    """
    # Initialize a counter for indexing 
    counter = 0
    
    # pre-initialize (for speed)
    complaint_feature_vecs = np.zeros((len(complaints),num_features), dtype='float32')  
    
    for complaint in complaints: # each complaint is made up of tokenized/cleaned/stopwords removed words
        complaint_feature_vecs[counter] = make_feature_vec(complaint, model, num_features)
        counter = counter + 1
    return complaint_feature_vecs

In [ ]:
# Tokenizing and vectorizing our Train_Val Complaints (80%)
clean_train_val_complaints = []
for complaint in X_train_val:
    clean_train_val_complaints.append(complaint_to_wordlist(complaint, True))

X_train_val_glove_features = get_avg_feature_vecs(clean_train_val_complaints, glove_model_50d, num_features)

# Tokenizing and vectorizing our Test Complaints (20%)
clean_test_complaints = []
for complaint in X_train_val:
    clean_test_complaints.append(complaint_to_wordlist(complaint, True))

X_test_glove_features = get_avg_feature_vecs(clean_test_complaints, glove_model_50d, num_features)

In [ ]:
## Run the X_train_val_word2vec_features into our defined function for scoring 

# Initialize Model Object
gnb = GaussianNB()
mnb = MultinomialNB()
logit = LogisticRegression(random_state=seed)
randomforest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
linearsvc = LinearSVC()

## We do NOT want these two. They take FOREVER to train AND predict
# knn = KNeighborsClassifier()  
# decisiontree = DecisionTreeClassifier(random_state=seed)

# to concat all models
results_cv_straitified_glove50d = pd.concat([
#     metric_cv_stratified(mnb, X_train_val_glove_features, y_train_val, 5, 'MultinomialNB_glove50d'),
     metric_cv_stratified(gnb, X_train_val_glove_features, y_train_val, 5, 'GaussianNB_glove50d'),
     metric_cv_stratified(logit, X_train_val_glove_features, y_train_val, 5, 'LogisticRegression_glove50d'),
     metric_cv_stratified(randomforest, X_train_val_glove_features, y_train_val, 5, 'RandomForest_glove50d'),
     metric_cv_stratified(linearsvc, X_train_val_glove_features, y_train_val, 5, 'LinearSVC_glove50d')
    ], axis=0).reset_index()

In [ ]:
# Saving Results into a DF
with open('results_cv_straitified_glove50d.pickle', 'wb') as to_write:
    pickle.dump(results_cv_straitified_glove50d, to_write)

In [ ]:
files.download('results_cv_straitified_glove50d.pickle')

In [ ]:
# Opening Results
with open('results_cv_straitified_glove50d.pickle', 'rb') as to_read:
    results_cv_straitified_glove50d = pickle.load(to_read)

In [ ]:
results_cv_straitified_glove50d

### Section 3.3.4: Using GloVe100d

In [ ]:
del glove_model_50d, results_cv_straitified_glove50d

In [ ]:
## Using pre-trained GloVe
# download from https://nlp.stanford.edu/projects/glove/

num_features = 100 # depends on the pre-trained model you are loading

glove_file = glove_dir = 'glove.6B.' + str(num_features) + 'd.txt'
w2v_output_file = 'glove.6B.' + str(num_features) + 'd.txt.w2v'

# The following utility converts file formats
gensim.scripts.glove2word2vec.glove2word2vec(glove_file, w2v_output_file)

# Now we can load it!
glove_model_100d = gensim.models.KeyedVectors.load_word2vec_format(w2v_output_file, binary=False)

# Pickle glove model so we don't have to do the above steps again and keep the damn glove.6b.50d in our folder
with open('glove_model_' + str(num_features) + 'd.pickle', 'wb') as to_write:
    pickle.dump(glove_model_100d, to_write)

In [ ]:
files.download('glove_model_' + str(num_features) + 'd.pickle')

In [ ]:
# Load pickled glove_model
with open('glove_model_100d.pickle', 'rb') as to_read:
    glove_model_100d = pickle.load(to_read)

In [ ]:
# For Train_Val Complaints (80%)
clean_train_val_complaints = []
for complaint in X_train_val:
    clean_train_val_complaints.append(complaint_to_wordlist(complaint, True))

X_train_val_glove_features = get_avg_feature_vecs(clean_train_val_complaints, glove_model_100d, num_features)

# For Test Complaints (20%)
clean_test_complaints = []
for complaint in X_train_val:
    clean_test_complaints.append(complaint_to_wordlist(complaint, True))

X_test_glove_features = get_avg_feature_vecs(clean_test_complaints, glove_model_100d, num_features)

In [ ]:
## Run the X_train_val_word2vec_features into our defined function for scoring 

# Initialize Model Object
gnb = GaussianNB()
mnb = MultinomialNB()
logit = LogisticRegression(random_state=seed)
randomforest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
linearsvc = LinearSVC()

## We do NOT want these two. They take FOREVER to train AND predict
# knn = KNeighborsClassifier()  
# decisiontree = DecisionTreeClassifier(random_state=seed)

# to concat all models
results_cv_straitified_glove100d = pd.concat([
#     metric_cv_stratified(mnb, X_train_val_glove_features, y_train_val, 5, 'MultinomialNB_glove50d'),
     metric_cv_stratified(gnb, X_train_val_glove_features, y_train_val, 5, 'GaussianNB_glove100d'),
     metric_cv_stratified(logit, X_train_val_glove_features, y_train_val, 5, 'LogisticRegression_glove100d'),
     metric_cv_stratified(randomforest, X_train_val_glove_features, y_train_val, 5, 'RandomForest_glove100d'),
     metric_cv_stratified(linearsvc, X_train_val_glove_features, y_train_val, 5, 'LinearSVC_glove100d')
    ], axis=0).reset_index()

In [ ]:
with open('results_cv_straitified_glove100d.pickle', 'wb') as to_write:
    pickle.dump(results_cv_straitified_glove100d, to_write)

In [ ]:
files.download('results_cv_straitified_glove100d.pickle')

In [ ]:
# Opening Results
with open('results_cv_straitified_glove100d.pickle', 'rb') as to_read:
    results_cv_straitified_glove100d = pickle.load(to_read)

In [ ]:
results_cv_straitified_glove100d

### Section 3.3.5: Using GloVe200d

In [ ]:
del glove_model_100d, results_cv_straitified_glove100d

In [ ]:
## Using pre-trained GloVe
# download from https://nlp.stanford.edu/projects/glove/

num_features = 200 # depends on the pre-trained model you are loading

glove_file = glove_dir = 'glove.6B.' + str(num_features) + 'd.txt'
w2v_output_file = 'glove.6B.' + str(num_features) + 'd.txt.w2v'

# The following utility converts file formats
gensim.scripts.glove2word2vec.glove2word2vec(glove_file, w2v_output_file)

# Now we can load it!
glove_model_200d = gensim.models.KeyedVectors.load_word2vec_format(w2v_output_file, binary=False)

# Pickle glove model so we don't have to do the above steps again and keep the damn glove.6b.50d in our folder
with open('glove_model_' + str(num_features) + 'd.pickle', 'wb') as to_write:
    pickle.dump(glove_model_200d, to_write)

In [ ]:
# Load pickled glove_model
with open('glove_model_200d.pickle', 'rb') as to_read:
    glove_model_200d = pickle.load(to_read)

In [ ]:
files.download('glove_model_' + str(num_features) + 'd.pickle')

In [ ]:
# For Train_Val Complaints (80%)
clean_train_val_complaints = []
for complaint in X_train_val:
    clean_train_val_complaints.append(complaint_to_wordlist(complaint, True))

X_train_val_glove_features = get_avg_feature_vecs(clean_train_val_complaints, glove_model_200d, num_features)

# Already run above
# # For Test Complaints (20%)
# clean_test_complaints = []
# for complaint in X_train_val:
#     clean_test_complaints.append(complaint_to_wordlist(complaint, True))

# X_test_glove_features = get_avg_feature_vecs(clean_test_complaints, glove_model_200d, num_features)

In [ ]:
## Run the X_train_val_word2vec_features into our defined function for scoring 

# Initialize Model Object
gnb = GaussianNB()
mnb = MultinomialNB()
logit = LogisticRegression(random_state=seed)
randomforest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
linearsvc = LinearSVC()

## We do NOT want these two. They take FOREVER to train AND predict
# knn = KNeighborsClassifier()  
# decisiontree = DecisionTreeClassifier(random_state=seed)

# to concat all models
results_cv_straitified_glove200d = pd.concat([
#     metric_cv_stratified(mnb, X_train_val_glove_features, y_train_val, 5, 'MultinomialNB_glove50d'),
     metric_cv_stratified(gnb, X_train_val_glove_features, y_train_val, 5, 'GaussianNB_glove200d'),
     metric_cv_stratified(logit, X_train_val_glove_features, y_train_val, 5, 'LogisticRegression_glove200d'),
     metric_cv_stratified(randomforest, X_train_val_glove_features, y_train_val, 5, 'RandomForest_glove200d'),
     metric_cv_stratified(linearsvc, X_train_val_glove_features, y_train_val, 5, 'LinearSVC_glove200d')
    ], axis=0).reset_index()

In [ ]:
with open('results_cv_straitified_glove200d.pickle', 'wb') as to_write:
    pickle.dump(results_cv_straitified_glove200d, to_write)

In [ ]:
files.download('results_cv_straitified_glove200d.pickle')

In [ ]:
# Opening Results
with open('results_cv_straitified_glove200d.pickle', 'rb') as to_read:
    results_cv_straitified_glove200d = pickle.load(to_read)

In [ ]:
results_cv_straitified_glove200d

### Section 3.3.6: Using GloVe300d

In [ ]:
del glove_model_200d, results_cv_straitified_glove200d

In [ ]:
## Using pre-trained GloVe
# download from https://nlp.stanford.edu/projects/glove/

num_features = 300 # depends on the pre-trained model you are loading

glove_file = glove_dir = 'glove.6B.' + str(num_features) + 'd.txt'
w2v_output_file = 'glove.6B.' + str(num_features) + 'd.txt.w2v'

# The following utility converts file formats
gensim.scripts.glove2word2vec.glove2word2vec(glove_file, w2v_output_file)

# Now we can load it!
glove_model_300d = gensim.models.KeyedVectors.load_word2vec_format(w2v_output_file, binary=False)

# Pickle glove model so we don't have to do the above steps again and keep the damn glove.6b.50d in our folder
with open('glove_model_' + str(num_features) + 'd.pickle', 'wb') as to_write:
    pickle.dump(glove_model_300d, to_write)

In [ ]:
files.download('glove_model_' + str(num_features) + 'd.pickle')

In [ ]:
# Load pickled glove_model
with open('glove_model_300d.pickle', 'rb') as to_read:
    glove_model_300d = pickle.load(to_read)

In [ ]:
# For Train_Val Complaints (80%)
clean_train_val_complaints = []
for complaint in X_train_val:
    clean_train_val_complaints.append(complaint_to_wordlist(complaint, True))

X_train_val_glove_features = get_avg_feature_vecs(clean_train_val_complaints, glove_model_300d, num_features)

# Already run above
# # For Test Complaints (20%)
# clean_test_complaints = []
# for complaint in X_train_val:
#     clean_test_complaints.append(complaint_to_wordlist(complaint, True))

# X_test_glove_features = get_avg_feature_vecs(clean_test_complaints, glove_model_200d, num_features)

In [ ]:
## Run the X_train_val_word2vec_features into our defined function for scoring 

# Initialize Model Object
gnb = GaussianNB()
mnb = MultinomialNB()
logit = LogisticRegression(random_state=seed)
randomforest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
linearsvc = LinearSVC()

## We do NOT want these two. They take FOREVER to train AND predict
# knn = KNeighborsClassifier()  
# decisiontree = DecisionTreeClassifier(random_state=seed)

# to concat all models
results_cv_straitified_glove300d= pd.concat([
#     metric_cv_stratified(mnb, X_train_val_glove_features, y_train_val, 5, 'MultinomialNB_glove50d'),
     metric_cv_stratified(gnb, X_train_val_glove_features, y_train_val, 5, 'GaussianNB_glove300d'),
     metric_cv_stratified(logit, X_train_val_glove_features, y_train_val, 5, 'LogisticRegression_glove300d'),
     metric_cv_stratified(randomforest, X_train_val_glove_features, y_train_val, 5, 'RandomForest_glove300d'),
     metric_cv_stratified(linearsvc, X_train_val_glove_features, y_train_val, 5, 'LinearSVC_glove300d')
    ], axis=0).reset_index()

In [ ]:
with open('results_cv_straitified_glove300d.pickle', 'wb') as to_write:
    pickle.dump(results_cv_straitified_glove300d, to_write)

In [ ]:
files.download('results_cv_straitified_glove300d.pickle')

In [ ]:
# Opening Results
with open('results_cv_straitified_glove300d.pickle', 'rb') as to_read:
    results_cv_straitified_glove300d = pickle.load(to_read)

In [ ]:
results_cv_straitified_glove300d

### Section 3.3.7: Using GoogleNews Word2Vec300d

In [ ]:
del glove_model_300d, results_cv_straitified_glove300d

In [ ]:
## Using pre-trained GoogleNews Word2Vec
# download from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

num_features = 300 # depends on the pre-trained model you are loading

# Path to where the word2vec file lives
google_vec_file = 'GoogleNews-vectors-negative300.bin'

# Load it!  This might take a few minutes...
word2vec_model_300d = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)
# it is just loading all the different weights (embedding) into python


# Pickle word2vec 300d model so we don't have to do the above steps again and keep the damn file in our folder
with open('word2vec_model_' + str(num_features) + 'd.pickle', 'wb') as to_write:
    pickle.dump(word2vec_model_300d, to_write)

In [ ]:
files.download('word2vec_model_' + str(num_features) + 'd.pickle')

In [ ]:
# Load pickled glove_model
with open('word2vec_model_300d.pickle', 'rb') as to_read:
    word2vec_model_300d = pickle.load(to_read)

In [ ]:
# For Train_Val Complaints (80%)
clean_train_val_complaints = []
for complaint in X_train_val:
    clean_train_val_complaints.append(complaint_to_wordlist(complaint, True))

X_train_val_glove_features = get_avg_feature_vecs(clean_train_val_complaints, word2vec_model_300d, num_features)

# Already run above
# # For Test Complaints (20%)
# clean_test_complaints = []
# for complaint in X_train_val:
#     clean_test_complaints.append(complaint_to_wordlist(complaint, True))

# X_test_glove_features = get_avg_feature_vecs(clean_test_complaints, glove_model_200d, num_features)

In [ ]:
## Run the X_train_val_word2vec_features into our defined function for scoring 

# Initialize Model Object
gnb = GaussianNB()
mnb = MultinomialNB()
logit = LogisticRegression(random_state=seed)
randomforest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
linearsvc = LinearSVC()

## We do NOT want these two. They take FOREVER to train AND predict
# knn = KNeighborsClassifier()  
# decisiontree = DecisionTreeClassifier(random_state=seed)

# to concat all models
results_cv_straitified_word2vec300d= pd.concat([
#     metric_cv_stratified(mnb, X_train_val_glove_features, y_train_val, 5, 'MultinomialNB_glove50d'),
     metric_cv_stratified(gnb, X_train_val_glove_features, y_train_val, 5, 'GaussianNB_word2vec300d'),
     metric_cv_stratified(logit, X_train_val_glove_features, y_train_val, 5, 'LogisticRegression_word2vec300d'),
     metric_cv_stratified(randomforest, X_train_val_glove_features, y_train_val, 5, 'RandomForest_word2vec300d'),
     metric_cv_stratified(linearsvc, X_train_val_glove_features, y_train_val, 5, 'LinearSVC_word2vec300d')
    ], axis=0).reset_index()

In [ ]:
with open('results_cv_straitified_word2vec300d.pickle', 'wb') as to_write:
    pickle.dump(results_cv_straitified_word2vec300d, to_write)

In [ ]:
files.download('results_cv_straitified_word2vec300d.pickle')

In [ ]:
# Opening Results
with open('results_cv_straitified_word2vec300d.pickle', 'rb') as to_read:
    results_cv_straitified_word2vec300d = pickle.load(to_read)

In [ ]:
results_cv_straitified_word2vec300d

In [ ]:
del word2vec_model_300d, results_cv_straitified_word2vec300d

### Section 3.3.8: Compiling Results

In [ ]:
results_compiled.columns

In [ ]:
results_compiled = pd.concat([results_cv_straitified_1gram,
                           results_cv_straitified_glove50d,
                           results_cv_straitified_glove100d,
                           results_cv_straitified_glove200d,
                           results_cv_straitified_glove300d,
                           results_cv_straitified_word2vec300d]).reset_index().drop(['level_0','index'],axis=1)

In [ ]:
with open('results_compiled.pickle', 'wb') as to_write:
    pickle.dump(results_compiled, to_write)

results_compiled

#### Sorting in Descending order by Accuracy

In [ ]:
# Sorting results to see which one gives us the best results
results_compiled_sorted_by_accuracy = results_compiled.sort_values(by='Accuracy', ascending=False)

with open('results_compiled_sorted_by_accuracy.pickle', 'wb') as to_write:
    pickle.dump(results_compiled_sorted_by_accuracy, to_write)

results_compiled_sorted_by_accuracy

In [ ]:
files.download('results_compiled_sorted_by_accuracy.pickle')

In [ ]:
# Retrieving the Model that provides the highest Accuracy
results_highest_accuracy = results_compiled[results_compiled.Accuracy == results_compiled.Accuracy.max()]

with open('results_highest_accuracy.pickle', 'wb') as to_write:
    pickle.dump(results_highest_accuracy, to_write)

results_highest_accuracy

In [ ]:
# We see from the above that LogisiticRegression and LinearSVC on 1-gram gives pretty good results.
# But LogReg is the best. So let's go with that!

# Now let us do the final test!!
# Pretty decent accuracy, actually.
# I want to try 2-gram but my computer just simply won't be able to handle it

# Section 4: Final Test - 80% Train/Test split

## Section 4.1: Splitting to 80% Train and 20% unseen data
#### Remember the 20% data we heldout earlier that we didn’t use in the cross-validation? Here comes the part where we use it as an estimate as to how well our model will perform on unseen data (i.e. how it would perform in real-life scenarios).

In [ ]:
# Split the data into X and y data sets
X, y = df.CONSUMER_COMPLAINT, df.PRODUCT_ID
print('X shape:', X.shape, 'y shape:', y.shape)

# For text classification, ALWAYS split data first before vectorizing.
# This is because you don't want to cheat by having features (words) from the test data already being inside your train data
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, 
                                                            test_size=0.2,   # 80% train/cv, 20% test
                                                            stratify=y,
                                                            random_state=seed)
print('X_train', X_train_val.shape)
print('y_train', y_train_val.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

In [ ]:
# tfidf1 = 1-gram only. 
tfidf1 = TfidfVectorizer(sublinear_tf=True, # set to true to scale the term frequency in logarithmic scale.
                        min_df=5,
                        stop_words='english')
X_train_val_tfidf1 = tfidf1.fit_transform(X_train_val).toarray()
X_test_tfidf1 = tfidf1.transform(X_test)

In [ ]:
# Loading our saved models
# Consider training on the whole data instead now?
# Pickle trained Model for use in Flask App
with open('fitted_tfidf_to_use.pickle', 'wb') as to_write:
    pickle.dump(tfidf1, to_write)
with open('fitted_tfidf_to_use.pickle', 'rb') as to_read:  # loading the fitted tfidf with our 80% trained data
    fitted_tfidf_to_use = pickle.load(to_read)

In [ ]:
# Initializing our chosen logreg model
logit = LogisticRegression()

# Fitting our model
logit_finalized = logit.fit(X_train_val_tfidf1, y_train_val)

# Obtaining prediction
y_pred = logit_finalized.predict(X_test_tfidf1)

In [ ]:
# Pickle trained Model for use in Flask App
with open('logit_finalized.pickle', 'wb') as to_write:
    pickle.dump(logit_finalized, to_write)
with open('logit_finalized.pickle', 'rb') as to_read:  # loading our model
    logit_finalized = pickle.load(to_read)

In [ ]:
# Create a function to calculate the error metrics, since we'll be doing this several times
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

def conf_matrix(actual, predicted):
    
    # Creates a dataframe of the ISSUE to their respective ISSUE_ID
    category_id_df = df[['PRODUCT', 'PRODUCT_ID']].drop_duplicates()

    # Dictionaries for future use. Creating our cheatsheets for what each encoded label represents.
    category_to_id = dict(category_id_df.values) # Creates a ISSUE: ISSUE_ID key-value pair
    id_to_category = dict(category_id_df[['PRODUCT_ID', 'PRODUCT']].values)  # Creates a ISSUE_ID: ISSUE key-value pair

    conf_mat = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(8,8))
    g= sns.heatmap(conf_mat, 
                   annot=True, annot_kws={"size":10},
                   cmap=plt.cm.Reds, square=True,
                   fmt='d',
                   xticklabels=category_id_df.ISSUE.values, 
                   yticklabels=category_id_df.ISSUE.values)
    
#     # Changing the size of the xticks and ytick labels
#     ax.set_yticklabels(g.get_yticklabels(), rotation=90, size=10);
#     ax.set_xticklabels(g.get_xticklabels(), size=10);
    
    # Changing axis orientation & setting titles
    ax.set_xlabel('Prediction', size=15)
    ax.set_ylabel('Actual', rotation=0, labelpad=40,size=15)

#     plt.title("CONFUSION MATRIX - {}\n".format(name), size=16);
    
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)

In [ ]:
'''with open('results_highest_accuracy.pickle', 'rb') as to_read:
    results_highest_accuracy = pickle.load(to_read)

results_highest_accuracy'''

# Score model
print("---------------------------------------------------------")
print("LogisticRegression (1-gram) 80% TRAIN/20% TEST SCORES:")
print("---------------------------------------------------------")
print('\n')
#print('Train/Cross-Validation Test Accuracy Score (also micro F1) for LogisticRegression: {:.4f}'.format(results_highest_accuracy.Accuracy.max()))
print('Final Test Accuracy Score (also micro F1) for LogisticRegression: {:.4f}'.format(accuracy_score(y_test, y_pred)))
print('\n')
print('Macro Precision Score for LogisticRegression: {:.4f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall Score for LogisticRegression: {:.4f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1 score = {:.4f}'.format(f1_score(y_test, y_pred, average='macro')))
print('\n')
print('Micro Precision Score for LogisticRegression: {:.4f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall Score for LogisticRegression: {:.4f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1 score = {:.4f}'.format(f1_score(y_test, y_pred, average='micro')))
print('\n')
print('Weighted Precision Score for LogisticRegression: {:.4f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall Score for LogisticRegression: {:.4f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1 score = {:.4f}'.format(f1_score(y_test, y_pred, average='weighted')))
print('\n')
print('Classification report for LogisticRegression (1-gram):\n {}'.format(classification_report(y_test, 
                                                                                             y_pred,
                                                                                             target_names=df_dc.PRODUCT.unique())))
print('Confusion Matrix for LogisticRegression (1-gram):\n'.format(conf_matrix(y_test, y_pred)))

In [ ]:
conf_matrix(y_test, y_pred)
plt.savefig('confusion_matrix', transparent=True)

## Section 5.1: Random Complaint 1

In [ ]:
# Using our models

complaint = """
omg where is my money
aosidnoiandnaiosdnasd This is bullshit!! where is my money!!!
I TRANSFERRED IT BUT IT JUST DISAPPEARED INTO THIN AIR.
WHAT KIND OF A BANK ARE YOU?
ARE YOU A BANK OR A THIEF?"""

# After fitting the tfidf vectorizor, then you can do transforms!
new_vectorized_complaint = fitted_tfidf_to_use.transform([complaint])

# Fitting vectorized complaint into model
y_customized_prediction = logit_finalized.predict(new_vectorized_complaint)
y_customized_prediction[0]

## Section 5.1: Random Complaint 2

In [ ]:
# Using our models

complaint_2 = """
I borrowed money from you guys but it seems that the interest terms are unclear?
Please shed some more light on this because i seem to be overcharged this month and I don't want to pay more than I should
"""

# After fitting the tfidf vectorizor, then you can do transforms!
new_vectorized_complaint_2 = fitted_tfidf_to_use.transform([complaint_2])

# Fitting vectorized complaint into model
y_customized_prediction = logit_finalized.predict(new_vectorized_complaint_2)
y_customized_prediction

In [ ]:
# Not bad! Our model actually does do a pretty good job at predicting

## Random Complaint 3 with no relevance at all

What about _completely_ irrelevant topics?

In [ ]:
# Using our models

complaint_3 = """I love macdonalds"""

# After fitting the tfidf vectorizor, then you can do transforms!
new_vectorized_complaint_3 = fitted_tfidf_to_use.transform([complaint_3])

# Fitting vectorized complaint into model
y_customized_prediction = logit_finalized.predict(new_vectorized_complaint_3)
y_customized_prediction

## Random Complaint 4 with no relevance at all

In [ ]:
# Using our models

complaint_4 = """Pair programming is not fun"""

# After fitting the tfidf vectorizor, then you can do transforms!
new_vectorized_complaint_4 = fitted_tfidf_to_use.transform([complaint_4])

# Fitting vectorized complaint into model
y_customized_prediction = logit_finalized.predict(new_vectorized_complaint_4)
y_customized_prediction

# Section 6: Highest Occuring Words in each column

In [ ]:
# tfidf2 = unigram and bigram
tfidf2 = TfidfVectorizer(sublinear_tf=True, # set to true to scale the term frequency in logarithmic scale.
                        min_df=5, 
                        ngram_range=(1,2), # we consider unigrams and bigrams
                        stop_words='english')

# We transform each complaint into a vector
features = tfidf2.fit_transform(df_dc.CONSUMER_COMPLAINT).toarray()
# Labelling our data
labels = df_dc.ISSUE_ID

print("Each of the %d complaints is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

In [ ]:
category_to_id

In [ ]:
# Finding the three most correlated terms with each of the product categories
N = 3
for Product, category_id in sorted(category_to_id.items()):
    features_chi2 = chi2(features, labels == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf2.get_feature_names())[indices]
    
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("\n==> %s:" %(Product))
    print("  * Most Correlated Unigrams are: %s" %(', '.join(unigrams[-N:])))
    print("  * Most Correlated Bigrams are: %s" %(', '.join(bigrams[-N:])))

## Productwise model building using Multiclassification methods on 'Issue' category
## To build productwise multiclassification model on issue category, selecting each product and build the model
## 1st Issue category model for Product='Debt collection' 

In [ ]:
def product_issue_cat_multiclf(df_product_and_complaint_reduced, product):
  global  product_nm, X, y, series, mask, X_train_val, X_test, y_train_val, y_test, tfidf1, X_train_val_tfidf1, tfidf2, X_train_val_tfidf2, X_test_tfidf1, X_test_tfidf2
  df = pd.DataFrame()
  product_nm = product
  df = df_product_and_complaint_reduced[['PRODUCT','ISSUE','COMPLAINT_ID','PRODUCT_ID','Modified_text']].rename(columns={'Modified_text':'CONSUMER_COMPLAINT'})
  df = df[(df['PRODUCT']==product_nm)]
  series = pd.value_counts(df.ISSUE)
  mask = (series/series.sum() * 100).lt(1)
  df['ISSUE'] = np.where(df['ISSUE'].isin(series[mask].index),'Other',df['ISSUE'])
  print(df[['PRODUCT','ISSUE']].value_counts(normalize=True)*100)
  # Subsetting dataframe into columns useful for our text multi-classification problem
  df = df[['ISSUE','CONSUMER_COMPLAINT']].copy()
  df.CONSUMER_COMPLAINT = df.CONSUMER_COMPLAINT.astype(str)
  df['ISSUE_ID'] = df['ISSUE'].factorize()[0]
  # Split the data into X and y data sets
  X, y = df.CONSUMER_COMPLAINT, df.ISSUE_ID
  # For text classification, ALWAYS split data first before vectorizing.
  # This is because you don't want to cheat by having features (words) from the test data already being inside your train data
  X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, 
                                                              test_size=0.2,   # 80% train/cv, 20% test
                                                              stratify=y,
                                                              random_state=seed)
  # tfidf1 = 1-gram only. 
  tfidf1 = TfidfVectorizer(sublinear_tf=True, # set to true to scale the term frequency in logarithmic scale.
                          min_df=5,
                          stop_words='english')

  X_train_val_tfidf1 = tfidf1.fit_transform(X_train_val).toarray()
  X_test_tfidf1 = tfidf1.transform(X_test)

  # tfidf2 = unigram and bigram
  '''tfidf2 = TfidfVectorizer(sublinear_tf=True, # set to true to scale the term frequency in logarithmic scale.
                            min_df=5, 
                            ngram_range=(1,2), # we consider unigrams and bigrams
                            stop_words='english')
    X_train_val_tfidf2 = tfidf2.fit_transform(X_train_val).toarray()
    X_test_tfidf2 = tfidf2.transform(X_test)'''
  
  # Initializing our chosen logreg model
  logit = LogisticRegression()

  # Fitting our model
  logit_finalized_1 = logit.fit(X_train_val_tfidf1, y_train_val)
  #logit_finalized_2 = logit.fit(X_train_val_tfidf2, y_train_val)
  # Obtaining prediction
  y_pred_1 = logit_finalized_1.predict(X_test_tfidf1)
  #y_pred_2 = logit_finalized_2.predict(X_test_tfidf2)
  #return X_test_tfidf1, X_test_tfidf2, logit_finalized_1, logit_finalized_2, y_pred_1, y_pred_2, y_test
  return X_test_tfidf1, logit_finalized_1, y_pred_1, y_test, df

In [ ]:
# Create a function to calculate the error metrics, since we'll be doing this several times
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

def conf_matrix(df_product_and_complaint_reduced, product, y_test, y_pred):
    global  product_nm
    df = pd.DataFrame()
    product_nm = product
    df = df_product_and_complaint_reduced[['PRODUCT','ISSUE','COMPLAINT_ID','PRODUCT_ID','Modified_text']].rename(columns={'Modified_text':'CONSUMER_COMPLAINT'})
    df = df[(df['PRODUCT']==product_nm)]
    series = pd.value_counts(df.ISSUE)
    mask = (series/series.sum() * 100).lt(1)
    df['ISSUE'] = np.where(df['ISSUE'].isin(series[mask].index),'Other',df['ISSUE'])
    df['ISSUE_ID'] = df['ISSUE'].factorize()[0]
    # Creates a dataframe of the ISSUE to their respective ISSUE_ID
    category_id_df = df[['ISSUE', 'ISSUE_ID']].drop_duplicates()

    # Dictionaries for future use. Creating our cheatsheets for what each encoded label represents.
    category_to_id = dict(category_id_df.values) # Creates a ISSUE: ISSUE_ID key-value pair
    id_to_category = dict(category_id_df[['ISSUE_ID', 'ISSUE']].values)  # Creates a ISSUE_ID: ISSUE key-value pair

    conf_mat = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(8,8))
    g= sns.heatmap(conf_mat, 
                   annot=True, annot_kws={"size":10},
                   cmap=plt.cm.Reds, square=True,
                   fmt='d',
                   xticklabels=category_id_df.ISSUE.values, 
                   yticklabels=category_id_df.ISSUE.values)
    
#     # Changing the size of the xticks and ytick labels
#     ax.set_yticklabels(g.get_yticklabels(), rotation=90, size=10);
#     ax.set_xticklabels(g.get_xticklabels(), size=10);
    
    # Changing axis orientation & setting titles
    ax.set_xlabel('Prediction', size=15)
    ax.set_ylabel('Actual', rotation=0, labelpad=40,size=15)

#     plt.title("CONFUSION MATRIX - {}\n".format(name), size=16);
    
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)

In [ ]:
'''Debt collection                dc
Mortgage                        mrg
Credit card                     cc
Loan                            ln
Credit reporting                cr
Checking or savings account     sa
Bank account or service         ba
Money transfers                 mt
Prepaid card                     pc
Other financial service           of'''

product = 'Debt collection'
prd_int = 'dc'
#X_test_tfidf1, X_test_tfidf2, logit_finalized_1, logit_finalized_2, y_pred_1, y_pred_2, y_test = product_issue_cat_multiclass(df_product_and_complaint_reduced,product)
X_test_tfidf1, logit_finalized_1, y_pred_1, y_test, df = product_issue_cat_multiclf(df_product_and_complaint_reduced,product)

# Pickle trained Model for use in Flask App
mdl_nm = prd_int+'_logit_finalized_1.pickle'
with open(mdl_nm, 'wb') as to_write:
    pickle.dump(logit_finalized_1, to_write)
with open(mdl_nm, 'rb') as to_read:  # loading our model
    logit_finalized_1 = pickle.load(to_read)

'''# Pickle trained Model for use in Flask App
mdl_nm = prd_int+'_logit_finalized_2.pickle'
with open(mdl_nm, 'wb') as to_write:
    pickle.dump(logit_finalized_2, to_write)
with open(mdl_nm, 'rb') as to_read:  # loading our model
    logit_finalized_2 = pickle.load(to_read)'''

# Loading our saved models
# Consider training on the whole data instead now?
# Pickle trained Model for use in Flask App
tfidf_nm = prd_int+'_fitted_tfidf_to_use.pickle'
with open(tfidf_nm, 'wb') as to_write:
    pickle.dump(tfidf1, to_write)
with open(tfidf_nm, 'rb') as to_read:  # loading the fitted tfidf with our 80% trained data
    fitted_tfidf_to_use = pickle.load(to_read)    

# Score model
print("---------------------------------------------------------")
print("LogisticRegression (1-gram) 80% TRAIN/20% TEST SCORES:")
print("---------------------------------------------------------")
print('\n')
#print('Train/Cross-Validation Test Accuracy Score (also micro F1) for LogisticRegression: {:.4f}'.format(results_highest_accuracy.Accuracy.max()))
print('Final Test Accuracy Score (also micro F1) for LogisticRegression: {:.4f}'.format(accuracy_score(y_test, y_pred_1)))
print('\n')
print('Macro Precision Score for LogisticRegression: {:.4f}'.format(precision_score(y_test, y_pred_1, average='macro')))
print('Macro Recall Score for LogisticRegression: {:.4f}'.format(recall_score(y_test, y_pred_1, average='macro')))
print('Macro F1 score = {:.4f}'.format(f1_score(y_test, y_pred_1, average='macro')))
print('\n')
print('Micro Precision Score for LogisticRegression: {:.4f}'.format(precision_score(y_test, y_pred_1, average='micro')))
print('Micro Recall Score for LogisticRegression: {:.4f}'.format(recall_score(y_test, y_pred_1, average='micro')))
print('Micro F1 score = {:.4f}'.format(f1_score(y_test, y_pred_1, average='micro')))
print('\n')
print('Weighted Precision Score for LogisticRegression: {:.4f}'.format(precision_score(y_test, y_pred_1, average='weighted')))
print('Weighted Recall Score for LogisticRegression: {:.4f}'.format(recall_score(y_test, y_pred_1, average='weighted')))
print('Weighted F1 score = {:.4f}'.format(f1_score(y_test, y_pred_1, average='weighted')))
print('\n')
print('Classification report for LogisticRegression (1-gram):\n {}'.format(classification_report(y_test, 
                                                                                             y_pred_1,
                                                                                             target_names=df.ISSUE.unique())))
print('Confusion Matrix for LogisticRegression (1-gram):\n'.format(conf_matrix(df_product_and_complaint_reduced, product, y_test, y_pred_1)))
plt.savefig('confusion_matrix', transparent=True)